In [1]:
from pathlib import Path
from itertools import product

import numpy as np
import pandas as pd
from scipy import stats

In [2]:
DATA_DIR = Path("../data/")

In [3]:
def read(path: Path, line_identifier="_10_"):
    with open(path, "r") as fh:
        lines = fh.readlines()
    
    data = [line.split() for line in lines if line_identifier in line]
    columns = ["instance", "env_seed", "agent_seed", "cost", "time"]
    types = [str, int, int, int, float]
    
    df = pd.DataFrame(data, columns=columns)
    for col, dtype in zip(columns, types):
        df[col] = df[col].astype(dtype)
        
    return df

def get_category(instance: str):
    if instance.startswith("C"):
        return "C"
    elif instance.startswith("RC"):
        return "RC"
    elif instance.startswith("R"):
        return "R"
    else:
        raise ValueError("Unknown instance name.")

def res2df(dir: Path):
    dfs = [] 
    for path in dir.rglob("*.txt"):
        df = read(path)
        
        *_, tw, arrival, method, fname = path.parts
        df["tw"] = tw
        df["method"] = method
        df["arrival"] = arrival
        df["category"] = df['instance'].apply(get_category)
    
        dfs.append(df)
    
    return pd.concat(dfs)

## Benchmark

In [4]:
df = res2df(DATA_DIR / "benchmark")
df = df[df['method'] != 'icd-rolling-horizon']
df.to_csv("../data/benchmark.csv", index=False)

best = df.groupby(["instance", "env_seed", "tw", "arrival"])["cost"].min().reset_index()
algs = df.groupby(["instance", "env_seed", "tw", "arrival", "method"])["cost"].min().unstack().reset_index()

best = best.sort_values(["instance", "env_seed", "tw", "arrival"])
algs = algs.sort_values(["instance", "env_seed", "tw", "arrival"])
algs["best"] = best["cost"]

In [5]:
# Sanity check that all results are in
null_rows = df[df.isnull().any(axis=1)]
assert null_rows.empty

df.groupby(['method']).count()

,instance,env_seed,agent_seed,cost,time,tw,arrival,category
method,,,,,,,,
hindsight,7200,7200,7200,7200,7200,7200,7200,7200
icd-dispatch-only,7200,7200,7200,7200,7200,7200,7200,7200
icd-double-threshold,7200,7200,7200,7200,7200,7200,7200,7200
icd-hamming-distance,7200,7200,7200,7200,7200,7200,7200,7200
icd-postpone-only,7200,7200,7200,7200,7200,7200,7200,7200
icd-rolling-horizon1,7200,7200,7200,7200,7200,7200,7200,7200


In [6]:
df = algs

# Compute gap for each method.
methods = [
    "icd-rolling-horizon1",
    "icd-dispatch-only",
    "icd-postpone-only",
    "icd-hamming-distance",
    "icd-double-threshold",
]
for method in methods:
    df[f"{method}_gap"] = (df[method] - df["best"]) / df["best"] * 100

df["category"] = df["instance"].apply(get_category)

In [7]:
df.groupby(["category", "arrival", "tw"])[[method + "_gap" for method in methods]].mean().round(2)

method                icd-rolling-horizon1_gap  icd-dispatch-only_gap  \
category arrival tw                                                     
C        hom     DL2                      1.80                   0.12   
                 DL4                      3.00                   1.41   
                 DL8                      4.28                   3.12   
                 TW2                      7.68                   6.39   
                 TW4                      7.29                   6.38   
                 TW8                      6.67                   5.92   
         uni     DL2                      2.20                   0.12   
                 DL4                      4.65                   2.63   
                 DL8                      6.77                   5.10   
                 TW2                      9.95                   9.91   
                 TW4                      9.98                  10.14   
                 TW8                      9.63                  10.15   
R        hom     DL2                      4.02                   2.66   
                 DL4                      8.66                   6.84   
                 DL8                     11.74                  10.20   
                 TW2                     13.38                  12.40   
                 TW4                     13.84                  13.07   
                 TW8                     13.52                  13.17   
         uni     DL2                      4.74                   2.73   
                 DL4                     10.20                   8.03   
                 DL8                     13.55                  12.13   
                 TW2                     14.15                  15.12   
                 TW4                     14.77                  15.86   
                 TW8                     14.63                  16.24   
RC       hom     DL2                      3.33                   2.29   
                 DL4                      7.79                   6.33   
                 DL8                     10.83                   9.76   
                 TW2                     12.84                  11.95   
                 TW4                     13.57                  12.95   
                 TW8                     13.07                  13.00   
         uni     DL2                      4.41                   2.49   
                 DL4                     10.47                   7.45   
                 DL8                     14.37                  11.60   
                 TW2                     14.21                  15.24   
                 TW4                     14.59                  16.19   
                 TW8                     14.37                  16.10   

method                icd-postpone-only_gap  icd-hamming-distance_gap  \
category arrival tw                                                     
C        hom     DL2                   0.16                      0.15   
                 DL4                   1.91                      1.29   
                 DL8                   4.45                      2.88   
                 TW2                   7.99                      6.48   
                 TW4                   8.11                      6.39   
                 TW8                   7.38                      5.97   
         uni     DL2                   0.16                      0.17   
                 DL4                   3.16                      2.63   
                 DL8                   6.39                      5.12   
                 TW2                  11.15                     10.25   
                 TW4                  11.14                     10.75   
                 TW8                  10.64                     10.77   
R        hom     DL2                   3.07                      2.46   
                 DL4                   8.00                      6.38   
                 DL8                  12.35              

In [8]:
# Groupby instance variation
res = df.groupby(["category", "arrival", "tw"])[[method + "_gap" for method in methods]].mean()
means = res.mean()

# Change column names
res.columns = ["RH", "DSHH", "Postpone", "Hamming", "Double"]

# Order as R, C and RC
res.sort_index(level=0, inplace=True, key=lambda x: x.map({"R": 0, "C":1, "RC":2}))

# Add dummy column for spacing and alignment
res.insert(0, "", "")

text = res.to_latex(float_format="%.2f")
text = text.replace("hom", "HOM")
text = text.replace("uni", "UNI")
print(text)

\begin{tabular}{llllrrrrr}
\toprule
   &     &     &    RH &  DSHH &  Postpone &  Hamming &  Double \\
category & arrival & tw &   &       &       &           &          &         \\
\midrule
R & HOM & DL2 &   &  4.02 &  2.66 &      3.07 &     2.46 &    2.57 \\
   &     & DL4 &   &  8.66 &  6.84 &      8.00 &     6.38 &    6.35 \\
   &     & DL8 &   & 11.74 & 10.20 &     12.35 &     9.86 &    9.73 \\
   &     & TW2 &   & 13.38 & 12.40 &     13.47 &    12.64 &   11.02 \\
   &     & TW4 &   & 13.84 & 13.07 &     14.29 &    13.49 &   11.76 \\
   &     & TW8 &   & 13.52 & 13.17 &     14.10 &    13.54 &   11.66 \\
   & UNI & DL2 &   &  4.74 &  2.73 &      3.01 &     2.78 &    2.69 \\
   &     & DL4 &   & 10.20 &  8.03 &      8.87 &     7.79 &    7.56 \\
   &     & DL8 &   & 13.55 & 12.13 &     13.17 &    12.19 &   11.41 \\
   &     & TW2 &   & 14.15 & 15.12 &     14.91 &    15.39 &   13.40 \\
   &     & TW4 &   & 14.77 & 15.86 &     15.91 &    16.48 &   14.46 \\
   &     & TW8 &   & 14.63 &

/var/folders/wh/bdq353fj58j61dwg1ksz7br40000gn/T/ipykernel_28951/4149941289.py:14: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  text = res.to_latex(float_format="%.2f")


In [9]:
means.round(2)

method
icd-rolling-horizon1_gap    9.58
icd-dispatch-only_gap       8.76
icd-postpone-only_gap       9.66
icd-hamming-distance_gap    8.83
icd-double-threshold_gap    8.06
dtype: float64

### Statistical tests

In [10]:
SIGNIFICANCE_LEVEL = 0.0125

In [11]:
groups = df.groupby(["category", "arrival", "tw", "env_seed"])[[method + "_gap" for method in methods]].mean()
variations = list(product(
    ["R", "C" ,"RC"],
    df["arrival"].unique(),
    df["tw"].unique(),
))
table_data = []

for category, arrival, tw in variations:
    data = groups.xs((category, arrival, tw)).dropna()
    baseline_method = data.mean().idxmin().removesuffix("_gap")
    baseline_data = data[baseline_method + "_gap"]
    p_vals = []

    for other_method in methods:
        if other_method != baseline_method:
            other_data = data[other_method + "_gap"]
            p_val = stats.ttest_ind(baseline_data, other_data).pvalue
            p_vals.append(p_val)
        else:
            p_vals.append(None)
            
    if all(p_val < SIGNIFICANCE_LEVEL for p_val in p_vals if p_val is not None):
        print(category, arrival, tw, baseline_method)

    table_data.append(p_vals)

R hom TW2 icd-double-threshold
R hom TW4 icd-double-threshold
R hom TW8 icd-double-threshold
R uni DL8 icd-double-threshold
R uni TW2 icd-double-threshold
C hom TW2 icd-double-threshold
C hom TW4 icd-double-threshold
C hom TW8 icd-double-threshold
C uni TW2 icd-double-threshold
C uni TW4 icd-double-threshold
RC hom TW2 icd-double-threshold
RC hom TW4 icd-double-threshold
RC hom TW8 icd-double-threshold
RC uni TW2 icd-double-threshold


In [12]:
# Groupby instance variation
res = df.groupby(["category", "arrival", "tw"])[[method + "_gap" for method in methods]].mean()
means = res.mean()

# Change column names
res.columns = ["RH", "DSHH", "Postpone", "Hamming", "Double"]

for idx, p_vals in zip(res.index, table_data):
    res.loc[idx] = p_vals

# Order as R, C and RC
res.sort_index(level=0, inplace=True, key=lambda x: x.map({"R": 0, "C":1, "RC":2}))

# Add dummy column for spacing and alignment
res.insert(0, "", "")

text = res.to_latex(float_format="%.3f")
text = text.replace("hom", "HOM")
text = text.replace("uni", "UNI")
text = text.replace("0.000", "\\textless 0.001")
text = text.replace("NaN", "-")
text = text.replace("\\cline{1-9} \\cline{2-9}", "\\midrule")
text = text.replace("\\cline{2-9}", "\\cmidrule(lr){2-9}")

print(text)

\begin{tabular}{llllrrrrr}
\toprule
   &     &     &    RH &  DSHH &  Postpone &  Hamming &  Double \\
category & arrival & tw &   &       &       &           &          &         \\
\midrule
R & HOM & DL2 &   & \textless 0.001 &   - &     0.014 &    0.044 &   0.911 \\
   &     & DL4 &   & \textless 0.001 & 0.132 &     \textless 0.001 &      - &   0.260 \\
   &     & DL8 &   & \textless 0.001 & 0.019 &     \textless 0.001 &      - &   0.075 \\
   &     & TW2 &   & \textless 0.001 & 0.001 &     \textless 0.001 &    \textless 0.001 &     - \\
   &     & TW4 &   & \textless 0.001 & \textless 0.001 &     \textless 0.001 &    \textless 0.001 &     - \\
   &     & TW8 &   & \textless 0.001 & \textless 0.001 &     \textless 0.001 &    \textless 0.001 &     - \\
   & UNI & DL2 &   & \textless 0.001 & 0.936 &     0.006 &    0.006 &     - \\
   &     & DL4 &   & \textless 0.001 & 0.452 &     \textless 0.001 &    0.497 &     - \\
   &     & DL8 &   & \textless 0.001 & 0.526 &     \textless 0.001 

/var/folders/wh/bdq353fj58j61dwg1ksz7br40000gn/T/ipykernel_28951/1885014337.py:17: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  text = res.to_latex(float_format="%.3f")


In [13]:
res.round(3)

RH   DSHH  Postpone  Hamming  Double
category arrival tw                                            
R        hom     DL2    0.000    NaN     0.014    0.044   0.911
                 DL4    0.000  0.132     0.000      NaN   0.260
                 DL8    0.000  0.019     0.000      NaN   0.075
                 TW2    0.000  0.001     0.000    0.000     NaN
                 TW4    0.000  0.000     0.000    0.000     NaN
                 TW8    0.000  0.000     0.000    0.000     NaN
         uni     DL2    0.000  0.936     0.006    0.006     NaN
                 DL4    0.000  0.452     0.000    0.497     NaN
                 DL8    0.000  0.526     0.000    0.429     NaN
                 TW2    0.000  0.000     0.000    0.000     NaN
                 TW4    0.002  0.000     0.000    0.000     NaN
                 TW8    0.017  0.000     0.000    0.000     NaN
C        hom     DL2    0.000  0.019     0.000      NaN   0.165
                 DL4    0.000  0.000     0.000    0.826     NaN
                 DL8    0.000  0.005     0.000    0.445     NaN
                 TW2    0.000  0.000     0.000    0.000     NaN
                 TW4    0.000  0.000     0.000    0.000     NaN
                 TW8    0.000  0.000     0.000    0.000     NaN
         uni     DL2    0.000  0.653     0.000    0.348     NaN
                 DL4    0.000  0.002     0.000    0.128     NaN
                 DL8    0.000  0.000     0.000    0.000     NaN
                 TW2    0.000  0.000     0.000    0.000     NaN
                 TW4    0.119  0.000     0.000    0.000     NaN
                 TW8    0.726  0.000     0.000    0.000     NaN
RC       hom     DL2    0.000  0.052     0.000      NaN   0.170
                 DL4    0.000  0.022     0.000      NaN   0.130
                 DL8    0.000  0.043     0.000    0.849     NaN
                 TW2    0.000  0.000     0.000    0.000     NaN
                 TW4    0.000  0.000     0.000    0.000     NaN
                 TW8    0.000  0.000     0.000    0.000     NaN
         uni     DL2    0.000  0.169     0.000    0.826     NaN
                 DL4    0.000  0.317     0.000    0.359     NaN
                 DL8    0.000  0.092     0.000    0.109     NaN
                 TW2    0.004  0.000     0.000    0.000     NaN
                 TW4      NaN  0.000     0.000    0.000   0.736
                 TW8      NaN  0.000     0.000    0.000   0.159